## Imports Here

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchvision.models import ResNet18_Weights

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

## Loading the CIFAR10 dataset

In [4]:
def load_data(batch_size=64):
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
    ])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

    return trainloader, testloader


## Loading a Pretrained Model - ResNet18

In [5]:
def build_model():
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
    model.fc = nn.Linear(512, 10)  # Adjust the output layer for 10 classes (CIFAR-10)
    return model.to(device)

## Training Loop

In [6]:
def train_model(model, trainloader, testloader, epochs=10, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        # Training loop
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(trainloader)

        # Evaluate the model on the test set
        test_loss, accuracy = evaluate_model(model, testloader)

        print(f"Epoch [{epoch+1}/{epochs}] | "
              f"Train Loss: {avg_train_loss:.4f} | "
              f"Test Loss: {test_loss:.4f} | "
              f"Test Accuracy: {accuracy:.2f}%")

    print('Finished Training')
    return model


def evaluate_model(model, testloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    test_loss = 0.0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_test_loss = test_loss / len(testloader)
    return avg_test_loss, accuracy

In [7]:
trainloader, testloader = load_data()
model = build_model()
trained_model = train_model(model, trainloader, testloader, epochs=50, lr=0.001)
torch.save(trained_model.state_dict(), "./models/resnet18_cifar10.pth")

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/50] | Train Loss: 1.0982 | Test Loss: 0.8732 | Test Accuracy: 69.75%
Epoch [2/50] | Train Loss: 0.8053 | Test Loss: 0.7469 | Test Accuracy: 75.21%
Epoch [3/50] | Train Loss: 0.7116 | Test Loss: 0.6936 | Test Accuracy: 76.70%
Epoch [4/50] | Train Loss: 0.6518 | Test Loss: 0.6425 | Test Accuracy: 78.40%
Epoch [5/50] | Train Loss: 0.6404 | Test Loss: 0.6755 | Test Accuracy: 77.58%
Epoch [6/50] | Train Loss: 0.5778 | Test Loss: 0.8223 | Test Accuracy: 75.04%
Epoch [7/50] | Train Loss: 0.5470 | Test Loss: 0.5763 | Test Accuracy: 80.30%
Epoch [8/50] | Train Loss: 0.5157 | Test Loss: 0.6582 | Test Accuracy: 79.39%
Epoch [9/50] | Train Loss: 0.5086 | Test Loss: 0.5398 | Test Accuracy: 82.28%
Epoch [10/50] | Train Loss: 0.4732 | Test Loss: 0.5439 | Test Accuracy: 81.27%
Epoch [11/50] | Train Loss: 0.4693 | Test Loss: 0.5226 | Test Accuracy: 82.10%
Epoch [12/50] | Train Loss: 0.4364 | Test Loss: 0.5073 | Test Ac